<a href="https://colab.research.google.com/github/marcogemaque/calculadora-aluguel-turma-set-20/blob/main/1-web-scraping/Awari_DS_Rent_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Webscraping and processing of file**

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

In [21]:
#Let's create a function to retrieve all the rent values
bairros = ['badu','barreto','baldeador','boa-viagem','camboinhas','centro','cubango','cantagalo','charitas',
           'engenho-do-mato','engenhoca','fonseca','fatima','gragoata','icarai','inga','itaipu',
           'ilha-da-conceicao','itacoatiara','jacare','jardim-fluminense','jardim-icarai',
           'largo-do-narradas','largo-da-batalha','maceio','maria-paula','maravista','matapaca',
           'pendotiba',"ponta-dareia",'piratininga','pe-pequeno','rio-do-ouro','santa-rosa',
           'serra-grande','sao-lourenco','santana','sao-domingos','sape','sao-francisco','tenente-jardim',
           'vital-brazil','varzea-das-mocas'] #List created to search in neighborhoods
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [22]:
def get_rents(x, hood, soup):
    #Function created to fetch rent, m², area and state and other important info
    tag_rent = soup.find_all('span',{"class":"price-tag-fraction"})
    list_rent = [r.text for r in tag_rent] #Create rent list
    tag_street = soup.find_all('h2',{"class":"ui-search-item__title ui-search-item__group__element"})
    list_street = [r.text for r in tag_street] #Create street rent

    if len(list_street) < len(list_rent):
        while len(list_street) < len(list_rent):
            list_street.append(" ")
    elif len(list_rent) < len(list_street):
        while len(list_rent) < len(list_street):
            list_rent.append("0")

    tag_area = soup.find_all('ul',{"class":'ui-search-card-attributes'})
    list_area = [r.text for r in tag_area] #Create area list
    if len(list_area) < len(list_rent):
        while len(list_area) < len(list_rent):
            list_area.append(0)
    elif len(list_rent) < len(list_area):
        while len(list_rent) < len(list_area):
            list_rent.append(0)

    df = pd.DataFrame(
        {'Rent':list_rent,
         'Street':list_street,
         'Area':list_area,
         'Hood':hood})
    
    df['Total-Area'] = df['Area'].str.split("[m]+")
    df['Total-Area'] = df['Total-Area'].str[0]
    df['Rooms'] = df['Area'].str.split("[construídos]+").str[1]
    df['Rooms'] = df['Rooms'].str[0]
    #df = df.drop(['Area'],axis=1) #Still not necessary
    df['Rooms'] = df['Rooms'].fillna(0)
    df['Rooms'] = pd.to_numeric(df['Rooms'])
    df['Rooms'] =df['Rooms'].replace(0,df.loc[df['Rooms'] > 0, 'Rooms'].mean())
    df['Total-Area'] = df['Total-Area'].replace("\,","",regex=True)
    df['Total-Area'] = pd.to_numeric(df['Total-Area'])
    df['Rent'] = df['Rent'].replace("\.","",regex=True)
    df['Rent'] = pd.to_numeric(df['Rent'])
    return df

In [23]:
df = pd.DataFrame(columns=["Rent","Street","Area","Hood","Total-Area","Rooms"])
url = 'https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi/'
for bairro in bairros:
    for pag in paginas:
        urls = url + "/{}/{}".format(bairro,pag)
        page = requests.get(urls)
        soup = BeautifulSoup(page.content,'html.parser')
        print('Currently scraping: {}'.format(urls))
        page = requests.get(urls)
        try:
            df_int = get_rents(page,bairro,soup)
            print(df_int.head())
            list_dfs = [df, df_int]
            df = pd.concat(list_dfs)
            sleep(1)
        except:
            print('exception')
            sleep(1)
            pass

Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/
   Rent                    Street  ... Total-Area Rooms
0  2100  Estrada Caetano Monteiro  ...         69     3
1  1650  Estrada Caetano Monteiro  ...         70     2
2  4800                            ...        300     5
3  3500  Estrada Caetano Monteiro  ...         90     3
4  1300              391 Bloco 06  ...         82     2

[5 rows x 6 columns]
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_49
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_97
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_145
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_193
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_

In [37]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1330 entries, 0 to 2
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rent        1330 non-null   object
 1   Street      1330 non-null   object
 2   Area        1330 non-null   object
 3   Hood        1330 non-null   object
 4   Total-Area  1328 non-null   object
 5   Rooms       1330 non-null   object
dtypes: object(6)
memory usage: 72.7+ KB


In [26]:
df_out.describe()

,Rent,Street,Area,Hood,Total-Area,Rooms
count,1330,1330,1330,1330,1328.0,1330.0
unique,202,478,371,34,225.0,37.0
top,1500,,60 m² construídos2 quartos,icarai,60.0,2.0
freq,80,451,69,336,91.0,476.0


In [73]:
df_out = df.copy()
#df_out.to_csv("consolidado.csv",sep=';',encoding='latin1')

In [74]:
#Fix the dataframe
df_out['Total-Area'] = df_out['Total-Area'].fillna(0)
df_out['Total-Area'] = pd.to_numeric(df_out['Total-Area'])
df_out['Rent'] = pd.to_numeric(df_out['Rent'])
df_out['Rooms'] = pd.to_numeric(df_out['Rooms'])
df_out = df_out.drop(['Area'],axis=1)

KeyError: ignored

In [61]:
df_out.to_csv("consolidado_v2.csv",sep=';',encoding='latin1')

## **EDA and ML of the files**

In [75]:
url = 'https://raw.githubusercontent.com/marcogemaque/calculadora-aluguel-turma-set-20/main/1-web-scraping/consolidado_v2.csv'
df = pd.read_csv(url,sep=';',encoding='latin1')

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1330 non-null   int64  
 1   Rent        1330 non-null   int64  
 2   Street      879 non-null    object 
 3   Hood        1330 non-null   object 
 4   Total-Area  1330 non-null   float64
 5   Rooms       1330 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 62.5+ KB
